In [ ]:
!pip install foolbox==2.4.0


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
from foolbox import zoo
%tensorflow_version 1.x

# download the model
model = zoo.get_model(url="https://github.com/bethgelab/cifar10_challenge")

In [212]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor()])

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=True, num_workers=2)

Extracting ./data/cifar-10-python.tar.gz to ./data


In [211]:
!rm -rf data

In [201]:
attack = foolbox.attacks.FGSM(model, distance=foolbox.distances.MeanSquaredDistance)
dataiter = iter(testloader)

orig_images = []
orig_labels = []

for i in range(10):
  image,label = dataiter.next()
  image = image.numpy()
  image = np.transpose(image,(0,2,3,1))
  image = image.reshape(32,32,3)
  orig_images.append(image)
  orig_labels.append(label.numpy()[0])
  
orig_images = np.array(orig_images)
orig_labels = np.array(orig_labels)

print ("Running Attack")
start_time = time.time()

# orig_images, orig_labels = foolbox.utils.samples(dataset='cifar10',batchsize=20, data_format='channels_last',bounds=(0,1))
adversarials = attack(orig_images, orig_labels, unpack=False)

end_time = time.time() - start_time
print ("Time taken: %.2f"%end_time)

adversarial_classes = np.asarray([a.adversarial_class for a in adversarials])
print(orig_labels)
print(adversarial_classes)

distances = np.asarray([a.distance.value for a in adversarials])
print("{:.1e}, {:.1e}, {:.1e}".format(distances.min(), np.median(distances), distances.max()))
print("{} of {} attacks failed".format(sum(adv.distance.value == np.inf for adv in adversarials), len(adversarials)))
print("{} of {} inputs misclassified without perturbation".format(sum(adv.distance.value == 0 for adv in adversarials), len(adversarials)))


# print ("Running Attack")
# start_time = time.time()
# attack = foolbox.attacks.FGSM(model, distance=foolbox.distances.MeanSquaredDistance)
# adversarials = attack(orig_images, orig_labels)
# end_time = time.time() - start_time

# print ("Time taken: %.2f"%end_time)


[8 2 7 3 1 3 5 6 4 5]
Running Attack
Time taken: 361.82
[8 2 7 3 1 3 5 6 4 5]
[7 4 8 2 8 4 3 4 6 3]
0.0e+00, 1.9e-09, 2.5e-08
0 of 10 attacks failed
2 of 10 inputs misclassified without perturbation


In [215]:
attack = foolbox.v1.attacks.LBFGSAttack(model, distance=foolbox.distances.MeanSquaredDistance)
dataiter = iter(testloader)

advs = []

print ("Running Attack")
start_time = time.time()
for i in range(1):
  image,label = dataiter.next()
  image = image.numpy()
  image = np.transpose(image,(0,2,3,1))
  image = image.reshape(32,32,3)
  label = label.numpy()[0]
  advs.append(attack(image, label))
  
# orig_images = np.array(orig_images)
# orig_labels = np.array(orig_labels)
end_time = time.time() - start_time
print ("Time taken: %.2f"%end_time)


# print ("Running Attack")
# start_time = time.time()
# attack = foolbox.attacks.FGSM(model, distance=foolbox.distances.MeanSquaredDistance)
# adversarials = attack(orig_images, orig_labels)
# end_time = time.time() - start_time

# print ("Time taken: %.2f"%end_time)

Running Attack
Time taken: 20.16


In [202]:
attack = foolbox.v1.attacks.FGSM(model, distance=foolbox.distances.MeanSquaredDistance)
dataiter = iter(testloader)
image,label = dataiter.next()

for i in range (1):
  image,label = dataiter.next()
  image = image.numpy()
  label = label.numpy()[0]
  print (label)
  image = np.transpose(image,(0,2,3,1))
  image = image.reshape(32,32,3)
  adversarials = attack(image, label)
  advs.append(adversarials)
  



6
